# Wellbeing Dashboard Python
## C - Picking Top Predictors & Building Final Models
This Jupyter Notebook evaluates over 500 impact indicators and tries to isolate the top 19 indicators that are strong predictors of well-being and are as well "controllable". Controllable is defined using intuition and ensuring that the indicators are something that entrepreneurs could potentially impact.

In [ ]:
# Doing all major library imports
import matplotlib.pyplot as plt
import scikitplot as skplt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import re

from sklearn import datasets, metrics
from sklearn.linear_model import LinearRegression, LogisticRegression,LogisticRegressionCV 
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
from matplotlib.colors import ListedColormap
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor, GradientBoostingClassifier, GradientBoostingRegressor, BaggingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVR

plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import scikitplot as skplt
from matplotlib.colors import ListedColormap
from sklearn.metrics import classification_report, confusion_matrix

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:.4f}'.format



In [ ]:
# Importing files that were generated in the earlier step (B - Imputing Missing Values)

witer = pd.read_csv ('../raw_data/iterated.csv') # with iterated / imputed values
woiter = pd.read_csv ('../raw_data/wo_iterated.csv') # without iterated / imputed values
print (witer.shape)
print (woiter.shape)

In [ ]:
# Checking out the data
witer.head(50)

## Step 1: Picking the top, most impactful indicators

In [ ]:
# Droping columns and rows based on thresholds that maximize the number of values we have avaible.
# Despite imputing values, some countries and indicators just didn't have enough information to make intelligent imputations.
# For this, I ran a gridsearch of sorts where I evaluated which thresholds save the most values
work = witer.dropna(axis=1, thresh=2600).dropna(axis=0, thresh=735).dropna(axis=1)
print (work.shape)

In [ ]:
# To mitigate risk of imputed values, I need to ensure that my test set is as robust as possible.
# To do that, I ensured that my test set is full of true values.
true_values = woiter[work.columns]
true_values = true_values[true_values.index.isin (work.index)]
print (true_values.shape)
print (true_values.isnull().sum().sum())
true_values.head()

In [ ]:
# Creating a new column called fill_level to evaluate how many true values each row has
true_values['fill_level'] = true_values.count(axis=1) / len(true_values.columns)

# Isolating the top 200 rows with the most true values / highest fill level
true_values_index = true_values.sort_values (by = 'fill_level', ascending = False).head(200).index
true_values = work[work.index.isin (true_values_index)]
print (true_values.shape)
true_values.head()

In [ ]:
# Isolating those rows that have more iterated values than true values
iterated_values = work[~work.index.isin (true_values.index)]
print (iterated_values.shape)
iterated_values.head()

In [ ]:
# Seeing what the year breakout is for our true values set
true_values.date.value_counts()

In [ ]:
# Creating a dev set which will consist partially of the true value data set and the iterated value data set
dev_true = true_values[true_values.date.isin (['2005','2014'])]
print (dev_true.shape)
dev_true.head()

In [ ]:
# Seeing what years are in the iterated value data set
iterated_values.date.value_counts()

In [ ]:
# Isolating some years to include in the dev set from the iterated values

dev_iterated = iterated_values[iterated_values.date.isin (['2010','2013', '1993', '2004'])]
print (dev_iterated.shape)

In [ ]:
# Combining the dev_true dataset and the dev_iterated dataset to get our final dev dataset
dev = pd.concat([dev_true,dev_iterated])
print (dev.shape)
dev.head()

In [ ]:
# Creating the test set that consists largely of true values
test = true_values[~true_values.index.isin (dev_true.index)]
print (test.shape)
test.head()

In [ ]:
# Creating the train set that consists largely of iterated values. This isn't ideal but it is more important for the test set to be real.
train = work[~(work.index.isin (test.index)) & ~(work.index.isin (dev.index))]
print (train.shape)
train.head()

In [ ]:
# Evaluating the shape of all our working sets for Step 1 of this process of isolating top indicators
print (work.shape)
print (train.shape)
print (dev.shape)
print (test.shape)

In [ ]:
# Isolating our target indicators

target_indicators = [
       'Mean years of schooling (years)',
       'Gross national income (GNI) per capita (2011 PPP $)',
       'Life expectancy at birth (years)',
       'Expected years of schooling (years)', 
]

target_indicators_all = [
       'Mean years of schooling (years)',
       'Life expectancy index', 
        'Income index',
       'Education index', 
        'Human Development Index (HDI)',
       'Gross national income (GNI) per capita (2011 PPP $)',
       'Life expectancy at birth (years)',
       'Expected years of schooling (years)', 
]

In [ ]:
# Predictors to drop because they are too closely related or not controllable.
# This was a manual and iterative process. Every time I ran a model, more of these indicators emerged as strong predictors...
# ...but were not controllable or were too closely related to the component factors of the well-being index (HDI)

predictors_to_drop = [
'GDP per capita, PPP (constant 2011 international $)',
'GDP per capita (2011 PPP $)',
'Estimated gross national income per capita, male (2011 PPP $)',
'GNI per capita, PPP (constant 2011 international $)',
'Estimated gross national income per capita, female (2011 PPP $)',
'GDP per person employed (constant 2011 PPP $)',
'GNI per capita, PPP (current international $)',
'Gross domestic product per capita, constant prices',
'Primary income receipts (BoP, current US$)',
'GDP per capita, PPP (current international $)',
'GNI per capita (constant 2010 US$)',
'GDP per capita (constant 2010 US$)',
'GNI, PPP (current international $)',
'GNI, PPP (constant 2011 international $)',
'GNI, Atlas method (current US$)',
'GNI per capita, Atlas method (current US$)',
'GNI per capita growth (annual %)',
'GNI growth (annual %)',
'GNI (current US$)',
'GNI (constant 2010 US$)',
'GDP, PPP (current international $)',
'GDP, PPP (constant 2011 international $)',
'GDP per unit of energy use (constant 2011 PPP $ per kg of oil equivalent)',
'Gross capital formation (annual % growth)',
'Gross capital formation (constant 2010 US$)',
'Gross capital formation (current US$)',
'Gross domestic product (GDP), total (2011 PPP $ billions)',
'Households and NPISHs Final consumption expenditure per capita (constant 2010 US$)',
'GDP per capita (current US$)',
'Price level ratio of PPP conversion factor (GDP) to market exchange rate',
'Gross savings (% of GDP)',
'Energy use (kg of oil equivalent) per $1,000 GDP (constant 2011 PPP)',
'Energy intensity level of primary energy (MJ/$2011 PPP GDP)',
'Electric power consumption (kWh per capita)',
'CO2 emissions (kg per 2010 US$ of GDP)',
'GDP per unit of energy use (PPP $ per kg of oil equivalent)',
'CO2 emissions (kg per 2011 PPP $ of GDP)',
'CO2 emissions (kg per PPP $ of GDP)',
'Carbon dioxide emissions (kg per 2011 PPP $ of GDP)',
'CO2 intensity (kg per kg of oil equivalent energy use)',
'Out-of-pocket expenditure per capita (current US$)',
'Domestic general government health expenditure (% of current health expenditure)',
'Domestic general government health expenditure per capita (current US$)',
'Carbon dioxide emissions, per capita (tonnes)',
'Mortality rate, under-5 (per 1,000 live births)',
'Mortality rate, female adult (per 1,000 people)',
'Mortality rate, male adult (per 1,000 people)',
'Mortality rate, infant (per 1,000 live births)_y',
'Mortality rate, neonatal (per 1,000 live births)',
'Mortality rate, under-five (per 1,000 live births)',
'Mortality rate, infant (per 1,000 live births)_x',
'Maternal mortality ratio (deaths per 100,000 live births)',
'Population with at least some secondary education (% ages 25 and older)',
'Population with at least some secondary education, female (% ages 25 and older)',
'School enrollment, secondary (% gross)',
'Population with at least some secondary education, male (% ages 25 and older)',
'GDP deflator (base year varies by country)',
'Current health expenditure per capita (current US$)',
'Out-of-pocket expenditure (% of current health expenditure)',
'Current health expenditure per capita, PPP (current international $)',
'Population ages 15-64 (% of total population)',
'Population ages 80 and above, male (% of male population)',
'Population ages 55-59, male (% of male population)',
'Current health expenditure per capita (current US$)',
'GDP deflator: linked series (base year varies by country)',
'PM2.5 air pollution, population exposed to levels exceeding WHO guideline value (% of total)',
'Reserves and related items (BoP, current US$)',
'Inflation, GDP deflator (annual %)',
'Population ages 30-34, female (% of female population)',
'Population ages 55-59, female (% of female population)',
'Population ages 35-39, female (% of female population)',
'Population, male (% of total population)',
'Age dependency ratio, old (% of working-age population)',
'Age dependency ratio, young (% of working-age population)',
'Agriculture, forestry, and fishing, value added (annual % growth)',
'Air transport, freight (million ton-km)',
'Air transport, registered carrier departures worldwide',
'Arable land (hectares)',
'Broad money growth (annual %)',
'CO2 emissions from electricity and heat production, total (% of total fuel combustion)',
'CO2 emissions from liquid fuel consumption (% of total)',
'CO2 emissions from liquid fuel consumption (kt)',
'CO2 emissions from residential buildings and commercial and public services (% of total fuel combustion)',
'CO2 emissions from transport (% of total fuel combustion)',
'Coefficient of human inequality',
'Current account balance',
'Current account balance (BoP, current US$)',
'Food production index (2004-2006 = 100)',
'Foreign direct investment, net inflows (% of GDP)_y',
'Population ages 00-04, female (% of female population)',
'Population ages 00-04, male (% of male population)',
'Population ages 0-14 (% of total population)',
'Population ages 0-14, female',
'Population ages 0-14, female (% of female population)',
'Population ages 0-14, male',
'Population ages 0-14, male (% of male population)',
'Population ages 0-14, total',
'Population ages 05-09, female (% of female population)',
'Population ages 10-14, female (% of female population)',
'Population ages 10-14, male (% of male population)',
'Population ages 15-19, female (% of female population)',
'Population ages 15-64, female',
'Population ages 15-64, male',
'Population ages 15-64, total',
'Population ages 15–64 (millions)',
'Population ages 25-29, female (% of female population)',
'Population ages 25-29, male (% of male population)',
'Population ages 30-34, male (% of male population)',
'Population ages 35-39, male (% of male population)',
'Population ages 40-44, female (% of female population)',
'Population ages 40-44, male (% of male population)',
'Population ages 45-49, female (% of female population)',
'Population ages 45-49, male (% of male population)',
'Population ages 50-54, female (% of female population)',
'Population ages 50-54, male (% of male population)',
'Population ages 60-64, female (% of female population)',
'Population ages 60-64, male (% of male population)',
'Population ages 65 and above (% of total population)',
'Population ages 65 and above, female',
'Population ages 65 and above, female (% of female population)',
'Population ages 65 and above, male',
'Population ages 65 and above, male (% of male population)',
'Population ages 65 and above, total',
'Population ages 65 and older (millions)',
'Population ages 65-69, female (% of female population)',
'Population ages 65-69, male (% of male population)',
'Population ages 70-74, female (% of female population)',
'Population ages 70-74, male (% of male population)',
'Population ages 75-79, female (% of female population)',
'Population ages 75-79, male (% of male population)',
'Population ages 80 and above, female (% of female population)',
'Population density (people per sq. km of land area)',
'Population growth (annual %)',
'Population under age 5 (millions)',
'Population, female',
'Population, male',
'Population, total',
'Renewable energy consumption (% of total final energy consumption)_y',
'Revenue',
'Population ages 05-09, male (% of male population)',
'Population, female (% of total population)',
'Population ages 20-24, male (% of male population)',
'Rural population growth (annual %)',
'Population ages 20-24, female (% of female population)',
'Population ages 15-19, male (% of male population)',
'Fertility rate, total (births per woman)',
'Prevalence of anemia among children (% of children under 5)',
'Agricultural raw materials imports (% of merchandise imports)',
'Capture fisheries production (metric tons)',
'Net imports [Imports - Exports - Bunkers] (petajoules)',
'Prevalence of anemia among pregnant women (%)', 
'Employment in agriculture, male (% of male employment) (modeled ILO estimate)',
'Immunization, measles (% of children ages 12-23 months)',
'Prevalence of anemia among women of reproductive age (% of women ages 15-49)',
'Prevalence of anemia among non-pregnant women (% of women ages 15-49)',
'Young age (0-14) dependency ratio (per 100 people ages 15-64)',
'Export unit value index (2000 = 100)',
'Old-age (65 and older) dependency ratio (per 100 people ages 15-64)',
'Infants lacking immunization, measles (% of one-year-olds)',
'Gross national expenditure deflator (base year varies by country)',
'General government final consumption expenditure (% of GDP)',
'School enrollment, primary, female (% gross)',
'Primary school starting age (years)',
'GDP (constant 2010 US$)',
'Inflation, average consumer prices',
'Gross domestic savings (current US$)',   
'Supply per capita (gigajoules)',
'Primary income payments (BoP, current US$)',
'Import volume index (2000 = 100)',
'Emissions per capita (metric tons of carbon dioxide)',
'Net primary income (BoP, current US$)',
'Final consumption expenditure (% of GDP)',
'Inflation, end of period consumer prices',
'Gross capital formation (% of GDP)_x',
'Gross capital formation (% of GDP)_y',
'Gross capital formation (annual % growth)',
'Gross capital formation (constant 2010 US$)',
'Gross capital formation (current US$)',
'Gross domestic product (GDP), total (2011 PPP $ billions)',
'Gross domestic product based on purchasing-power-parity (PPP) share of world total',
'Gross domestic product corresponding to fiscal year, current prices',
'Gross domestic product per capita, constant prices',
'Gross domestic product per capita, current prices',
'Gross domestic product, constant prices',
'Gross domestic product, current prices',
'Gross domestic product, deflator',
'Gross domestic savings (% of GDP)',
'Gross domestic savings (current US$)',
'Gross fixed capital formation (% of GDP)_x',
'Gross fixed capital formation (% of GDP)_y',
'Gross fixed capital formation (constant 2010 US$)',
'Gross fixed capital formation (current US$)',
'Gross intake ratio in first grade of primary education, total (% of relevant age group)',
'Gross national expenditure (% of GDP)',
'Gross national expenditure (constant 2010 US$)',
'Gross national expenditure (current US$)',
'Gross national expenditure deflator (base year varies by country)',
'Gross national income (GNI) per capita (2011 PPP $)',
'Gross savings (% of GDP)',
'Gross savings (% of GNI)',
'Gross savings (current US$)',
'Households and NPISHs Final consumption expenditure (annual % growth)',
'Households and NPISHs Final consumption expenditure (constant 2010 US$)',
'Households and NPISHs Final consumption expenditure (current US$)',
'Households and NPISHs Final consumption expenditure per capita (constant 2010 US$)',
'Households and NPISHs Final consumption expenditure per capita growth (annual %)',
'Households and NPISHs Final consumption expenditure, PPP (constant 2011 international $)',
'Households and NPISHs final consumption expenditure (% of GDP)',
'Domestic general government health expenditure (% of current health expenditure)',
'Domestic general government health expenditure (% of general government expenditure)',
'Domestic general government health expenditure per capita (current US$)',
'Domestic general government health expenditure (% of GDP)',
'Number of under-five deaths',
'Overall loss in HDI due to inequality (%)',
'Median age (years)',
'Human Development Index (HDI), male',
'Human Development Index (HDI), female',
'Life expectancy at birth, total (years)',
'Life expectancy at birth, male (years)',
'Life expectancy at birth, female (years)',
'Mean years of schooling, male (years)',
'Mean years of schooling, female (years)',
'Expected years of schooling, male (years)',
'Expected years of schooling, female (years)',
'Inequality-adjusted education index',
'Inequality in life expectancy (%)',
'Inequality-adjusted income index',
'Inequality-adjusted HDI (IHDI)',
'Inequality-adjusted life expectancy index',
'Gender Inequality Index (GII)'
]



### Wellbeing Metric - Human Development Index (HDI)
After evaluating a few different types of metrics, the Wellbeing metric I ended up using was the Human Development Index (HDI). It is not perfect but is pretty holistic / straight-forward and covers healthcare, education and income. For more details on how it is calculated, please go here: http://hdr.undp.org/sites/default/files/hdr2019_technical_notes.pdf

In [ ]:
# Testing the HDI construction
life_expectancy = test['Life expectancy at birth (years)'].values[0]
print(life_expectancy)
expected_years = test['Expected years of schooling (years)'].values[0]
print (expected_years)
mean_years = test['Mean years of schooling (years)'].values[0]
print (mean_years)
gnipp = np.log(test['Gross national income (GNI) per capita (2011 PPP $)'].values[0])
print (np.exp(gnipp))
print (gnipp)

# Creating a function to calculate the HDI based on inputs)

def hdi_calculator (gnipp, life_expectancy, mean_years, expected_years):
    health_index = (life_expectancy - 20) / (85 - 20)
    expected_years_index = (expected_years - 0) / (18 - 0)
    mean_years_index = (mean_years - 0) / (15 - 0)
    education_index = (mean_years_index + expected_years_index) / 2
    #income_index = (np.log(gnipp) - np.log (100)) / (np.log(75000) - np.log (100))
    income_index = (gnipp - np.log (100)) / (np.log(75000) - np.log (100))
    hdi = (health_index * education_index * income_index)**(1/3)
    return hdi

hdi_calculator(gnipp, life_expectancy, mean_years, expected_years)

In [ ]:
# Building a model to isolate top predictors

# Creating the predictor matrix for train, dev and test sets
X_train = train.drop(columns = predictors_to_drop + target_indicators_all + ['country_code' , 'country', 'date'])
X_dev = dev.drop(columns = predictors_to_drop + target_indicators_all + ['country_code' , 'country', 'date'])
X_test = test.drop(columns = predictors_to_drop + target_indicators_all + ['country_code' , 'country', 'date'])

# For Timeseries Cross Validation. Since this data had time-bound data, this is an additional cross val test...
# ...to ensure that I am not very off
unique_years = work.date.unique()
X = work.drop(columns = predictors_to_drop + target_indicators_all + ['country_code' , 'country'])

# Initiating a StandardScaler object
scaler_X = StandardScaler()

### Model Election
At this point, I ran literally hundreds of different types of models trying to predict components of HDI (Mean Years of School, Expected Years of Schooling, GNIPP, Life Expectancy) as well trying to predict HDI as a final metric. 

Remember, in this step what was important was to able to isolate those indicators which were the strongest, least related and most controllable. It was important to be able to see feature importance / coefficient weights so I could only use model types that gave me those (Linear Regression, SVR Regression, Decision Tree Regressors including Boosting and Bagging Regressors). In the end, Linear Regression with Lasso regularlization was the most meaningful. That's the only one I show in this jupyter notebook for simplicitiy's sake.

In [ ]:
#HDI Impact Checker - Lasso

# Isolating the target variables
target = 'Human Development Index (HDI)'

y_train_hdi = train[target]
y_dev_hdi = dev[target]
y_test_hdi = test[target]
y = work[['date'] + [target]]

# Best model after Grid searching and evaluating many models
model_hdi = LassoCV(alphas=np.logspace(-4, 4, 100), cv=5, fit_intercept=True, max_iter=10000, tol = 0.01)

# Building a pipe to make the process more efficient
pipe_hdi = Pipeline(steps=[
                       ('standardize', scaler_X),
                       ('model', model_hdi)])

# Fitting the model
pipe_hdi.fit (X_train,y_train_hdi)

# Putting the predictions in a dataframe
pred_hdi = pd.DataFrame(pipe_hdi.predict (X_dev))

# Kfold cross validation
kf = KFold(n_splits=5, shuffle=True, random_state=1)
scores_shuffled = cross_val_score(pipe_hdi, X_train, y_train_hdi, cv=kf)

# Time_series cross validation (since this data had time-bound data, this is an additional cross val test to ensure...
# ... that I am not very off.)
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

ts_scores_train = []
ts_scores_test = []

for train_index, test_index in tscv.split(unique_years):
    X_train_ts = X[X.date.isin (unique_years[train_index])]
    X_test_ts = X[X.date.isin (unique_years[test_index])]
    X_train_ts.pop ('date')
    X_test_ts.pop ('date')
    
    y_train_ts = y[y.date.isin (unique_years[train_index])]
    y_test_ts = y[y.date.isin (unique_years[test_index])]
    y_train_ts.pop ('date')
    y_test_ts.pop ('date')
    
    ts_scores_train.append(pipe_hdi.score (X_train_ts, y_train_ts))
    ts_scores_test.append(pipe_hdi.score (X_test_ts, y_test_ts))

# Printing all ther results
print("\nKfold Cross Validation:")
print("Shuffled Cross-validated scores:", scores_shuffled)
print("Mean of Shuffled Cross-validated scores:", scores_shuffled.mean())

print("\nTime Series Cross Validation:")
print("Train Time-Series Cross Val mean score:", np.mean(ts_scores_train))
print("Test Time-Series Cross Val mean score:", np.mean(ts_scores_test))

print("\nTraining Score:", pipe_hdi.score(X_train,y_train_hdi))
print("Dev Score:", pipe_hdi.score(X_dev,y_dev_hdi))
print("Test Score:", pipe_hdi.score(X_test,y_test_hdi))

print ("\nRoot of Mean Squared Error:", mean_squared_error (y_dev_hdi,pred_hdi)**0.5)

print ("\nNegative Values:", np.sum ((pred_hdi <= 0).values*1))

# Building a dataframe for coefficients to isolate which ones are the most impactful
df_coef = pd.DataFrame(list(zip(X_train.columns, pipe_hdi.named_steps['model'].coef_)))
df_coef.columns = ['predictors', 'coefficients']
df_coef['coef_abs'] = df_coef.coefficients.abs()
df_coef.sort_values (by='coef_abs', ascending = False, inplace=True)
df_coef[df_coef.coef_abs>0]

In [ ]:
# Exporting these coefficients to a csv so that I can manually adjust and look for onces that are most impactful
# Used MS Excel for this because it was a small dataset and more flexible for manual analysis
df_coef.to_csv('../raw_data/picking_predictors.csv', index=False)

## Step 2: Using top selected indicators (19) to build best model to predict HDI

In [ ]:
# Based on the analysis done in Excel, these were top 19 indicators that were picked based on prediction power,...
# ... controllability and relatedness (lack of)

predictors_again = [
'Inequality in education (%)',
'School enrollment, primary (% gross)',
'Domestic general government health expenditure per capita, PPP (current international $)',
'Agriculture, forestry, and fishing, value added (% of GDP)',
'Adolescent fertility rate (births per 1,000 women ages 15-19)',
'Pupil-teacher ratio, primary',
'Tuberculosis incidence (per 100,000 people)',
'Secondary education, general pupils (% female)',
'Out-of-pocket expenditure per capita, PPP (current international $)',
'Inequality in income (%)',
'PM2.5 pollution, population exposed to levels exceeding WHO Interim Target-3 value (% of total)',
'Forest area (% of land area)',
'Mobile cellular subscriptions (per 100 people)',
'International inbound tourists (thousands)',
'Government expenditure on education, total (% of GDP)',
'Rural population with access to electricity (%)',
'Unemployment, total (% of labour force)',
'Corruption Perception Index',
'Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)',
 ]

In [ ]:
# Now that we have the indicators, we have to rebuild the train, dev and test dataset
# The reason to rebuild these datasets is because we have lesser indicators, we should have more rows to work with

work_log = witer[['country_code', 'country', 'date'] + predictors_again + target_indicators_all].dropna()
print (work_log.shape)
work_log.head()

In [ ]:
# Removing any negative values that might have been caused by imputation
work_log['negative_vals'] = work_log[work_log[predictors_again] < 0].count(axis=1) >= 1
work_log = work_log[work_log.negative_vals == False]
work_log.pop('negative_vals')
print (work_log.shape)
work_log.head()

In [ ]:
# Taking a deeper look at the indicators
work_log.describe()

In [ ]:
# Analyzing skew in the predictors so we can smooth the data as needed
work_log[predictors_again].skew()

In [ ]:
# Logging columns that have a skew greater than 1
logged_columns = []
for predictor in predictors_again:
    if np.abs(work_log[predictor].skew()) >=1:
        if np.abs(np.log(work_log[predictor]).skew()) < np.abs(work_log[predictor].skew()):
            work_log[predictor] = np.log(work_log[predictor])
            logged_columns.append(predictor)

# As the output shows, the data is much less skewed after logging the relevant columns 
work_log[predictors_again].skew()

In [ ]:
# Recreating the test / train / dev sets based on true values and iterated values as before
true_values = woiter[work_log.columns].dropna()
true_values = work_log[work_log.index.isin (true_values.index)]
print (true_values.shape)
true_values.head()

In [ ]:
iterated_values = work_log[~work_log.index.isin (true_values.index)]
print (iterated_values.shape)
iterated_values.head()

In [ ]:
true_values.date.value_counts()

In [ ]:
dev_true = true_values[true_values.date.isin (['2010','2012', '2014'])]
print (dev_true.shape)
dev_true.head()

In [ ]:
iterated_values.date.value_counts()

In [ ]:
dev_iterated = iterated_values[iterated_values.date.isin (['2017', '2013', '2011'])]
print (dev_iterated.shape)

In [ ]:
dev_log = pd.concat([dev_true,dev_iterated])
print (dev_log.shape)
dev_log.head()

In [ ]:
test_log = true_values[~true_values.index.isin (dev_true.index)]
print (test_log.shape)
test_log.head()

In [ ]:
train_log = work_log[~(work_log.index.isin (test_log.index)) & ~(work_log.index.isin (dev_log.index))]
print (train_log.shape)
train_log.head()

In [ ]:
# Ensuring the train, dev and test sets are all aligned and there are no duplicates
print (train_log.shape)
print (dev_log.shape)
print (test_log.shape)
print (work_log.shape)

In [ ]:
# Creating predictor matrices for all the train, dev and test datasets
X_trimmed_train = train_log[predictors_again]
X_trimmed_dev = dev_log[predictors_again]
X_trimmed_test = test_log[predictors_again]
X_trimmed = work_log[predictors_again + ['date']]

In [ ]:
# Ensuring there are no null values
X_trimmed_train.isnull().sum()

### Model Selection & Design:
For the final algorithm structure, the idea was predict the components of HDI (Mean Years of School, Expected Years of Schooling, GNIPP, Life Expectancy), and then use those predictions to calculate HDI based on the formula as per the UNDP formulation (http://hdr.undp.org/sites/default/files/hdr2019_technical_notes.pdf).

The goal was to eventually transfer this model to Tableau so I could create a toggle-able dashboard. Since I didnot have Tableau Server, I could not use TabPy and was only able to use Linear Regression. Why? Because Linear Regression gives me coefficients (and basically a formula) which I could then take to Tableau and use the model in the form of equations. 

Again, I toyed with many types of linear regression models and gridsearched all the four separate models of the four component indicators I was trying to predict to arrive at the best models which are presented below.

In [ ]:
# Model for GNI PP - LASSO
target = 'Gross national income (GNI) per capita (2011 PPP $)'

y_train_gni = np.log(train_log[target])
y_dev_gni = np.log(dev_log[target])
y_test_gni = np.log(test_log[target])
y = work_log[['date'] + [target]]
y[target] = np.log(y[target].values)

model_gni = LassoCV(alphas=np.logspace(-4, 4, 100), cv=5, eps=10, fit_intercept=True,
                         max_iter=100, n_alphas=10, n_jobs=None,
                         normalize=False, positive=False, precompute='auto',
                         random_state=None, selection='cyclic', tol=0.001,
                         verbose=False)

pipe_gni = Pipeline(steps=[
                       ('standardize', scaler_X),
                       ('model', model_gni)])

pipe_gni.fit (X_trimmed_train,y_train_gni)

pred_gni = pd.DataFrame(pipe_gni.predict (X_trimmed_dev))

#kfold cross validation
kf = KFold(n_splits=5, shuffle=True, random_state=1)
scores_shuffled = cross_val_score(pipe_gni, X_trimmed_train, y_train_gni, cv=kf)

#time_series cross validation
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

ts_scores_train = []
ts_scores_test = []

for train_index, test_index in tscv.split(unique_years):
    X_trimmed_train_ts = X_trimmed[X_trimmed.date.isin (unique_years[train_index])]
    X_trimmed_test_ts = X_trimmed[X_trimmed.date.isin (unique_years[test_index])]
    X_trimmed_train_ts.pop ('date')
    X_trimmed_test_ts.pop ('date')
    
    y_train_ts = y[y.date.isin (unique_years[train_index])]
    y_test_ts = y[y.date.isin (unique_years[test_index])]
    y_train_ts.pop ('date')
    y_test_ts.pop ('date')
    
    ts_scores_train.append(pipe_gni.score (X_trimmed_train_ts, y_train_ts))
    ts_scores_test.append(pipe_gni.score (X_trimmed_test_ts, y_test_ts))

print("\nKfold Cross Validation:")
print("Shuffled Cross-validated scores:", scores_shuffled)
print("Mean of Shuffled Cross-validated scores:", scores_shuffled.mean())

print("\nTime Series Cross Validation:")
print("Train Time-Series Cross Val mean score:", np.mean(ts_scores_train))
print("Test Time-Series Cross Val mean score:", np.mean(ts_scores_test))

print("\nTraining Score:", pipe_gni.score(X_trimmed_train,y_train_gni))
print("Dev Score:", pipe_gni.score(X_trimmed_dev,y_dev_gni))
print("Test Score:", pipe_gni.score(X_trimmed_test,y_test_gni))

print ("\nRoot of Mean Squared Error:", mean_squared_error (y_dev_gni,pred_gni)**0.5)

print ("\nNegative Values:", np.sum ((pred_gni <= 0).values*1))

# Building a dataframe for coefficients
df_coef = pd.DataFrame(list(zip(X_trimmed_train.columns, pipe_gni.named_steps['model'].coef_)))
df_coef.columns = ['predictors', 'coefficients']
df_coef['coef_abs'] = df_coef.coefficients.abs()
df_coef.sort_values (by='coef_abs', ascending = False, inplace=True)
df_coef[df_coef.coef_abs>0]

In [ ]:
# Model for Life Expectancy - LASSO
target = 'Life expectancy at birth (years)'

y_train_le = train_log[target]
y_dev_le = dev_log[target]
y_test_le = test_log[target]
y = work_log[['date'] + [target]]

model_le = LassoCV(alphas=np.logspace(-4, 4, 100), cv=5, fit_intercept=True, max_iter=10000, tol = 0.01)

pipe_le = Pipeline(steps=[
                       ('standardize', scaler_X),
                       ('model', model_le)])

pipe_le.fit (X_trimmed_train,y_train_le)

pred_le = pd.DataFrame(pipe_le.predict (X_trimmed_dev))

#kfold cross validation
kf = KFold(n_splits=5, shuffle=True, random_state=1)
scores_shuffled = cross_val_score(pipe_le, X_trimmed_train, y_train_le, cv=kf)

#time_series cross validation
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

ts_scores_train = []
ts_scores_test = []

for train_index, test_index in tscv.split(unique_years):
    X_trimmed_train_ts = X_trimmed[X_trimmed.date.isin (unique_years[train_index])]
    X_trimmed_test_ts = X_trimmed[X_trimmed.date.isin (unique_years[test_index])]
    X_trimmed_train_ts.pop ('date')
    X_trimmed_test_ts.pop ('date')
    
    y_train_ts = y[y.date.isin (unique_years[train_index])]
    y_test_ts = y[y.date.isin (unique_years[test_index])]
    y_train_ts.pop ('date')
    y_test_ts.pop ('date')
    
    ts_scores_train.append(pipe_le.score (X_trimmed_train_ts, y_train_ts))
    ts_scores_test.append(pipe_le.score (X_trimmed_test_ts, y_test_ts))

print("\nKfold Cross Validation:")
print("Shuffled Cross-validated scores:", scores_shuffled)
print("Mean of Shuffled Cross-validated scores:", scores_shuffled.mean())

print("\nTime Series Cross Validation:")
print("Train Time-Series Cross Val mean score:", np.mean(ts_scores_train))
print("Test Time-Series Cross Val mean score:", np.mean(ts_scores_test))

print("\nTraining Score:", pipe_le.score(X_trimmed_train,y_train_le))
print("Dev Score:", pipe_le.score(X_trimmed_dev,y_dev_le))
print("Test Score:", pipe_le.score(X_trimmed_test,y_test_le))

print ("\nRoot of Mean Squared Error:", mean_squared_error (y_dev_le,pred_le)**0.5)

print ("\nNegative Values:", np.sum ((pred_le <= 0).values*1))

# Building a dataframe for coefficients
df_coef = pd.DataFrame(list(zip(X_trimmed_train.columns, pipe_le.named_steps['model'].coef_)))
df_coef.columns = ['predictors', 'coefficients']
df_coef['coef_abs'] = df_coef.coefficients.abs()
df_coef.sort_values (by='coef_abs', ascending = False, inplace=True)
df_coef[df_coef.coef_abs>0]

In [ ]:
# Model for Mean years of schooling - LASSO
target = 'Mean years of schooling (years)'

y_train_mys = train_log[target]
y_dev_mys = dev_log[target]
y_test_mys = test_log[target]
y = work_log[['date'] + [target]]

model_mys = LassoCV(alphas=np.logspace(-4, 4, 100), cv=5, fit_intercept=True, max_iter=10000, tol = 0.01)

pipe_mys = Pipeline(steps=[
                       ('standardize', scaler_X),
                       ('model', model_mys)])

pipe_mys.fit (X_trimmed_train,y_train_mys)

pred_mys = pd.DataFrame(pipe_mys.predict (X_trimmed_dev))

#kfold cross validation
kf = KFold(n_splits=5, shuffle=True, random_state=1)
scores_shuffled = cross_val_score(pipe_mys, X_trimmed_train, y_train_mys, cv=kf)

#time_series cross validation
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

ts_scores_train = []
ts_scores_test = []

for train_index, test_index in tscv.split(unique_years):
    X_trimmed_train_ts = X_trimmed[X_trimmed.date.isin (unique_years[train_index])]
    X_trimmed_test_ts = X_trimmed[X_trimmed.date.isin (unique_years[test_index])]
    X_trimmed_train_ts.pop ('date')
    X_trimmed_test_ts.pop ('date')
    
    y_train_ts = y[y.date.isin (unique_years[train_index])]
    y_test_ts = y[y.date.isin (unique_years[test_index])]
    y_train_ts.pop ('date')
    y_test_ts.pop ('date')
    
    ts_scores_train.append(pipe_mys.score (X_trimmed_train_ts, y_train_ts))
    ts_scores_test.append(pipe_mys.score (X_trimmed_test_ts, y_test_ts))

print("\nKfold Cross Validation:")
print("Shuffled Cross-validated scores:", scores_shuffled)
print("Mean of Shuffled Cross-validated scores:", scores_shuffled.mean())

print("\nTime Series Cross Validation:")
print("Train Time-Series Cross Val mean score:", np.mean(ts_scores_train))
print("Test Time-Series Cross Val mean score:", np.mean(ts_scores_test))

print("\nTraining Score:", pipe_mys.score(X_trimmed_train,y_train_mys))
print("Dev Score:", pipe_mys.score(X_trimmed_dev,y_dev_mys))
print("Test Score:", pipe_mys.score(X_trimmed_test,y_test_mys))

print ("\nRoot of Mean Squared Error:", mean_squared_error (y_dev_mys,pred_mys)**0.5)

print ("\nNegative Values:", np.sum ((pred_mys <= 0).values*1))

# Building a dataframe for coefficients
df_coef = pd.DataFrame(list(zip(X_trimmed_train.columns, pipe_mys.named_steps['model'].coef_)))
df_coef.columns = ['predictors', 'coefficients']
df_coef['coef_abs'] = df_coef.coefficients.abs()
df_coef.sort_values (by='coef_abs', ascending = False, inplace=True)
df_coef[df_coef.coef_abs>0]

In [ ]:
# Model for Expected years of schooling - LASSO
target = 'Expected years of schooling (years)'

y_train_eys = train_log[target]
y_dev_eys = dev_log[target]
y_test_eys = test_log[target]
y = work_log[['date'] + [target]]

model_eys = LassoCV(alphas=np.logspace(-4, 4, 100), cv=5, fit_intercept=True, max_iter=10000, tol = 0.01)

pipe_eys = Pipeline(steps=[
                       ('standardize', scaler_X),
                       ('model', model_eys)])

pipe_eys.fit (X_trimmed_train,y_train_eys)

pred_eys = pd.DataFrame(pipe_eys.predict (X_trimmed_dev))

#kfold cross validation
kf = KFold(n_splits=5, shuffle=True, random_state=1)
scores_shuffled = cross_val_score(pipe_eys, X_trimmed_train, y_train_eys, cv=kf)

#time_series cross validation
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

ts_scores_train = []
ts_scores_test = []

for train_index, test_index in tscv.split(unique_years):
    X_trimmed_train_ts = X_trimmed[X_trimmed.date.isin (unique_years[train_index])]
    X_trimmed_test_ts = X_trimmed[X_trimmed.date.isin (unique_years[test_index])]
    X_trimmed_train_ts.pop ('date')
    X_trimmed_test_ts.pop ('date')
    
    y_train_ts = y[y.date.isin (unique_years[train_index])]
    y_test_ts = y[y.date.isin (unique_years[test_index])]
    y_train_ts.pop ('date')
    y_test_ts.pop ('date')
    
    ts_scores_train.append(pipe_eys.score (X_trimmed_train_ts, y_train_ts))
    ts_scores_test.append(pipe_eys.score (X_trimmed_test_ts, y_test_ts))

print("\nKfold Cross Validation:")
print("Shuffled Cross-validated scores:", scores_shuffled)
print("Mean of Shuffled Cross-validated scores:", scores_shuffled.mean())

print("\nTime Series Cross Validation:")
print("Train Time-Series Cross Val mean score:", np.mean(ts_scores_train))
print("Test Time-Series Cross Val mean score:", np.mean(ts_scores_test))

print("\nTraining Score:", pipe_eys.score(X_trimmed_train,y_train_eys))
print("Dev Score:", pipe_eys.score(X_trimmed_dev,y_dev_eys))
print("Test Score:", pipe_eys.score(X_trimmed_test,y_test_eys))

print ("\nRoot of Mean Squared Error:", mean_squared_error (y_dev_eys,pred_eys)**0.5)

print ("\nNegative Values:", np.sum ((pred_eys <= 0).values*1))

# Building a dataframe for coefficients
df_coef = pd.DataFrame(list(zip(X_trimmed_train.columns, pipe_eys.named_steps['model'].coef_)))
df_coef.columns = ['predictors', 'coefficients']
df_coef['coef_abs'] = df_coef.coefficients.abs()
df_coef.sort_values (by='coef_abs', ascending = False, inplace=True)
df_coef[df_coef.coef_abs>0]

In [ ]:
# Testing final models on how they predict HDI versus actual HDI
gni_predictions = pipe_gni.predict (X_trimmed_test)
le_predictions = pipe_le.predict (X_trimmed_test)
mys_predictions = pipe_mys.predict (X_trimmed_test)
eys_predictions = pipe_eys.predict (X_trimmed_test)

hdi_predicted = []
for i in range(len(gni_predictions)):
    hdi = hdi_calculator (gni_predictions[i],le_predictions[i],mys_predictions[i],eys_predictions[i])
    hdi_predicted.append(hdi)
hdi_predicted = np.nan_to_num (hdi_predicted)

hdi_test = test_log['Human Development Index (HDI)']

# R2 score for predicted HDIs based on our models and actual HDIs
r2_score(hdi_test, hdi_predicted)

In [ ]:
# Testing final models on how they predict HDI versus actual HDI only for true values
test_clean = woiter[['country_code','country', 'date'] + predictors_again + target_indicators_all].dropna()
for col in logged_columns:
    test_clean[col] = np.log(test_clean[col])

print(test_clean.shape)
test_clean.head()

In [ ]:
X_test_clean = test_clean[predictors_again]
print (X_test_clean.shape)
X_test_clean.head()

In [ ]:
# Testing final models on how they predict HDI versus actual HDI only for true values
gni_predictions = pipe_gni.predict (X_test_clean)
le_predictions = pipe_le.predict (X_test_clean)
mys_predictions = pipe_mys.predict (X_test_clean)
eys_predictions = pipe_eys.predict (X_test_clean)

hdi_predicted = []
for i in range(len(gni_predictions)):
    hdi = hdi_calculator (gni_predictions[i],le_predictions[i],mys_predictions[i],eys_predictions[i])
    hdi_predicted.append(hdi)
hdi_predicted = np.nan_to_num (hdi_predicted)

hdi_test = test_clean['Human Development Index (HDI)']

# R2 score for predicted HDIs based on our models and actual HDIs only for true values (excluding iterated values)
r2_score(hdi_test, hdi_predicted)

### Step 3: Exporting model to Tableau through CSVs

In [ ]:
# Now that the models are built, we need to export the data and the model coefficients into CSVs to take to Tableau

# Building a function to create coefficient tables for export
def coef_table_creator_lasso (fitted_model, predictor_df, name_str):
    coef_tbl = pd.DataFrame(list(zip((name_str+predictor_df.columns), fitted_model.coef_)), columns = ['Constant', 'Value'])
    intercept_tbl = pd.DataFrame({'Constant' : [name_str+'Intercept'], 
               'Value' : [fitted_model.intercept_]})
    final_table = pd.concat ([coef_tbl, intercept_tbl])
    return final_table


In [ ]:
gni_tbl = coef_table_creator_lasso (pipe_gni.named_steps['model'], X_trimmed_train, 'GNI_')
le_tbl = coef_table_creator_lasso (pipe_le.named_steps['model'], X_trimmed_train, 'LE_')
mys_tbl = coef_table_creator_lasso (pipe_mys.named_steps['model'],  X_trimmed_train, 'MYS_')
eys_tbl = coef_table_creator_lasso (pipe_eys.named_steps['model'],  X_trimmed_train, 'EYS_')
std_tbl = pd.DataFrame(list(zip(('STD_'+ X_trimmed_train.columns), scaler_X.scale_)), columns = ['Constant', 'Value'])
mn_tbl = pd.DataFrame(list(zip(('MN_'+ X_trimmed_train.columns), scaler_X.mean_)), columns = ['Constant', 'Value'])

# Merging all the tables into one final table for export
model_table_one = pd.concat([gni_tbl, le_tbl,mys_tbl,eys_tbl, std_tbl, mn_tbl]).reset_index(drop=True)
model_table_one

In [ ]:
# Exporting final model table for Tableau
model_table_one.index = model_table_one.Constant
model_table_one.pop('Constant')
model_table_one.T.to_csv ('../tableau_data/model_table_one.csv')

In [ ]:
# Preparing clean (true values) data for Tableau export
data_clean = woiter[['country_code', 'country', 'date'] + predictors_again + target_indicators_all + ['High Inc Low Inc Class']]
print(data_clean.shape)
data_clean.head()

In [ ]:
# Preparing iterated data for Tableau export
data_iterated = witer[['country_code', 'country', 'date'] + predictors_again + target_indicators_all + ['High Inc Low Inc Class']]
data_iterated['negative_vals'] = data_iterated[data_iterated[predictors_again + target_indicators_all] < 0].count(axis=1) >= 1
data_iterated = data_iterated[data_iterated.negative_vals == False]
data_iterated.pop('negative_vals')
print(data_iterated.shape)
data_iterated.head()

In [ ]:
# Preparing logged data for Tableau export
data_log = work_log.copy()
print (data_log.shape)
data_log.head()

In [ ]:
# Exporting data for Tableau
data_clean.to_csv ('../tableau_data/data_clean.csv', index=False)
data_iterated.to_csv ('../tableau_data/data_iterated.csv', index=False)
data_log.to_csv ('../tableau_data/data_log.csv', index=False)

In [ ]:
# Creating Model to generate feature importance for Tableau
target = 'Human Development Index (HDI)'

y_train_hdi_imp = train_log[target]
y_dev_hdi_imp = dev_log[target]
y_test_hdi_imp = test_log[target]
y = work_log[['date'] + [target]]

model_hdi_imp = LassoCV(alphas=np.logspace(-4, 4, 100), cv=5, fit_intercept=True, max_iter=10000, tol = 0.01)

pipe_hdi_imp = Pipeline(steps=[
                       ('standardize', scaler_X),
                       ('model', model_hdi_imp)])

pipe_hdi_imp.fit (X_trimmed_train,y_train_hdi_imp)

pred_hdi_imp = pd.DataFrame(pipe_hdi_imp.predict (X_trimmed_dev))

#kfold cross validation
kf = KFold(n_splits=5, shuffle=True, random_state=1)
scores_shuffled = cross_val_score(pipe_hdi_imp, X_trimmed_train, y_train_hdi_imp, cv=kf)

#time_series cross validation
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

ts_scores_train = []
ts_scores_test = []

for train_index, test_index in tscv.split(unique_years):
    X_trimmed_train_ts = X_trimmed[X_trimmed.date.isin (unique_years[train_index])]
    X_trimmed_test_ts = X_trimmed[X_trimmed.date.isin (unique_years[test_index])]
    X_trimmed_train_ts.pop ('date')
    X_trimmed_test_ts.pop ('date')
    
    y_train_ts = y[y.date.isin (unique_years[train_index])]
    y_test_ts = y[y.date.isin (unique_years[test_index])]
    y_train_ts.pop ('date')
    y_test_ts.pop ('date')
    
    ts_scores_train.append(pipe_hdi_imp.score (X_trimmed_train_ts, y_train_ts))
    ts_scores_test.append(pipe_hdi_imp.score (X_trimmed_test_ts, y_test_ts))

print("\nKfold Cross Validation:")
print("Shuffled Cross-validated scores:", scores_shuffled)
print("Mean of Shuffled Cross-validated scores:", scores_shuffled.mean())

print("\nTime Series Cross Validation:")
print("Train Time-Series Cross Val mean score:", np.mean(ts_scores_train))
print("Test Time-Series Cross Val mean score:", np.mean(ts_scores_test))

print("\nTraining Score:", pipe_hdi_imp.score(X_trimmed_train,y_train_hdi_imp))
print("Dev Score:", pipe_hdi_imp.score(X_trimmed_dev,y_dev_hdi_imp))
print("Test Score:", pipe_hdi_imp.score(X_trimmed_test,y_test_hdi_imp))

print ("\nRoot of Mean Squared Error:", mean_squared_error (y_dev_hdi_imp,pred_hdi_imp)**0.5)

print ("\nNegative Values:", np.sum ((pred_hdi_imp <= 0).values*1))

# Building a dataframe for coefficients
df_coef = pd.DataFrame(list(zip(X_trimmed_train.columns, pipe_hdi_imp.named_steps['model'].coef_)))
df_coef.columns = ['predictors', 'coefficients']
df_coef['coef_abs'] = df_coef.coefficients.abs()
df_coef.sort_values (by='coef_abs', ascending = False, inplace=True)
df_coef[df_coef.coef_abs>0]

In [ ]:
# Creating a relative_imp column to calculate relative importance to be used in Tableau
df_coef['relative_imp'] = df_coef.coef_abs.apply(lambda x: x/(np.sum(df_coef.coef_abs)))
df_coef

In [ ]:
# Exporting feature importance file for Tableau usage
df_coef.to_csv ('../tableau_data/feature_importance.csv', index=False)

## Time for Tableau!

The final step of this project is to take the model and the data into Tableau to build an interactive dashboard that can actually be used by non-coding users to potentially make better hypothesis on what impacts well-being and how.